In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import defaultdict, OrderedDict

In [3]:
vit = pd.read_csv("../../ensemble_subs/ViT_ensemble/csv")
beiter = pd.read_csv("../../ensemble_subs/beiter_ensemble.csv")
beiter10 = pd.read_csv("../../ensemble_subs/voting_ensemble.csv")
resnet = pd.read_csv("../../ensemble_subs/resnet18_ensemble.csv")
densenet161 = pd.read_csv("../../ensemble_subs/densenet161_ensemble.csv")
densenet121 = pd.read_csv("../../ensemble_subs/densenet121_ensemble.csv")

In [34]:
resnet = resnet.drop("Unnamed: 0", axis = 1)
resnet.head()

,img,1,2,3,4,5
0,test_2004.jpg,46,78,17,65,33
1,test_1714.jpg,31,49,23,74,50
2,test_3171.jpg,57,30,74,61,67
3,test_2733.jpg,28,6,37,15,12
4,test_113.jpg,64,10,2,6,23


In [36]:
densenet161 = densenet161.drop("Unnamed: 0", axis = 1)
densenet161.head()

,img,1,2,3,4,5
0,test_2485.jpg,17,41,28,14,71
1,test_1426.jpg,62,48,26,37,8
2,test_4488.jpg,31,18,66,68,53
3,test_5742.jpg,65,1,39,17,53
4,test_5772.jpg,61,15,7,49,71


In [46]:
final = defaultdict(lambda : defaultdict(float))

In [47]:
models = [(vit, 0.651), (beiter, 0.694), (beiter10, 0.710), (resnet, 0.401), (densenet161, 0.629), (densenet121, 0.404)]

In [48]:
def prob_to_log(row):
    for i, val in enumerate(row):
        if i:
            val = eval(val)
            row[i] = (np.log(val[0]), val[1]) 
        
    return row
    
# beiter10 = beiter10.apply(prob_to_log, axis = 1)

In [49]:
proba = False

In [50]:
if proba:
    for model, acc in models:
        for row in model.itertuples():   
            for rank in range(3):
                if type(row[2 + rank]) != type(()):
                    proba, label = eval(row[2 + rank])
                else:
                    proba, label = row[2 + rank]
                final[row[1]][label] += proba

else:
    for model, acc in models:
        for pred in model.itertuples():
            for rank in range(3):
                final[pred[1]][pred[2 + rank]] += ((5 - rank) / 2) * acc

In [51]:
top1 = {k : max(v, key = v.get) for k, v in final.items()}

In [52]:
test_df = pd.Series(top1)

test_df = test_df.to_frame().reset_index().rename({"index": "img_name", 0:"label"}, axis = 1)

test_df.head(10)

,img_name,label
0,test_1.jpg,63
1,test_10.jpg,45
2,test_100.jpg,12
3,test_1000.jpg,15
4,test_1001.jpg,10
5,test_1002.jpg,14
6,test_1003.jpg,41
7,test_1004.jpg,28
8,test_1005.jpg,15
9,test_1006.jpg,47


In [53]:
test_df.to_csv("../../submissions/manual_ensemble_all.csv", index=False)